In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import statistics

class Indivisual():
    def __init__(self, x):
        self.sex = np.random.choice(["m", "f"], p=[x, (1-x)])
        self.sexratio = x
        self.age = 0
        
    def deth(self): # 死亡処理(0が死，1が生を意味する)
        if self.age == 10: # 10歳で確実に死ぬ
            return 0
        else:
            return np.random.choice([0, 1], p=[0.1, 0.9]) # 1/10の確率で死ぬ
        
    def spawn(self, male): # 卵を1年で10個産む
        egg = []
        for i in range(10):
            egg.append(nextGen(self.sexratio, male))
            
        return egg
    
    def aging(self): # 歳を取る
        self.age = self.age + 1
    
    

def nextGen(female, male):
    ave = 0
    nextgen = -1
    
    # 両親の性比強度の平均
    ave = (female+male)/2
    # 平均ave、分散0.1の正規分布から次世代の値を決定
    while nextgen < 0 or nextgen >= 1:
        nextgen = np.random.normal(ave, 0.1)
    
    return nextgen
    
def copulation(population):
    male_population = []
    female_population = []
    newgen = []
    # 計算量抑制のため，集団の1/3のみを選出，そこから交尾可能か探る
    selected_population = random.sample(population, round(len(population)/3))
    
    for i in range(len(selected_population)):
        if selected_population[i].sex == "f":
            female_population.append(selected_population[i])
        else:
            male_population.append(selected_population[i])

    if len(female_population) >= len(male_population):
        cop_i = len(male_population)
    else:
        cop_i = len(female_population)
        
    for i in range(cop_i): # 交尾させる雌雄の抽出と交配
        cop_female = female_population.pop(random.randrange(len(female_population)))
        cop_male = male_population.pop(random.randrange(len(male_population)))
        newgen.append(cop_female.spawn(cop_male.sexratio))
        
    return newgen
        
def initialize(x, firstsize): # 初期化設定
    population = []

    for i in range(firstsize):
        population.append(Indivisual(x))

    copulation(population) # 1世代目の作出
    return population

def tiktok(population):
    for popu in population[:]: 
        popu.aging() # 歳を取らせてる
        if popu.deth() == 0: # もし死んでいたら削除
            index = population.index(popu)
            del population[index]

    newegg = copulation(population)
    if newegg is not None:
        for i in range(len(newegg)):
            for j in range(10): 
                population.append(Indivisual(newegg[i][j]))
                
def populationControl(population, limit):
    while len(population) > int(limit):
        index = random.randrange(len(population))
        del population[index]

def main(population, iteration, limit):
    b = []
    S_x = []
    summarize = []

    for i in range(int(iteration / 10)):
        a = 0
        b.append(a+(i*10))
    
    for i in tqdm(range(iteration)):
        tiktok(population)
        populationControl(population, limit)
        
        if i in b:
            for i in range(len(population)):
                S_x.append(population[i].sexratio)
            summarize.append((statistics.median(S_x)))
                

    plt.plot(b, summarize)
    plt.title("sex ratio")
    plt.xlabel("Generations")
    plt.grid(True)


In [ ]:
main(initialize(0.2,50), 100, 200)